# Information Retrieval

## Creating Documents

The Reuters-21578 files are in [SGML format](http://kdd.ics.uci.edu/databases/reuters21578/README.txt). We need to split them into individual documents. There should be 1000.

In [3]:
from collections import defaultdict
from bs4 import BeautifulSoup

import re
import math
import string

In [6]:
with open('./data/reut2-000.sgm') as f:
    corpus = f.read()

In [13]:
soup = BeautifulSoup(corpus, 'html.parser')
allArticles = soup.find_all('reuters')

articles = [] # stories with a body
documents = []
for a in allArticles:
    if a.body:
        articles.append(a)
        documents.append(re.split('\W+', a.body.string))

In [11]:
re.split('\s', 'asdf asdf\nasdf')

['asdf', 'asdf', 'asdf']

In [14]:
documents[0]


['Showers',
 'continued',
 'throughout',
 'the',
 'week',
 'in',
 'the',
 'Bahia',
 'cocoa',
 'zone',
 'alleviating',
 'the',
 'drought',
 'since',
 'early',
 'January',
 'and',
 'improving',
 'prospects',
 'for',
 'the',
 'coming',
 'temporao',
 'although',
 'normal',
 'humidity',
 'levels',
 'have',
 'not',
 'been',
 'restored',
 'Comissaria',
 'Smith',
 'said',
 'in',
 'its',
 'weekly',
 'review',
 'The',
 'dry',
 'period',
 'means',
 'the',
 'temporao',
 'will',
 'be',
 'late',
 'this',
 'year',
 'Arrivals',
 'for',
 'the',
 'week',
 'ended',
 'February',
 '22',
 'were',
 '155',
 '221',
 'bags',
 'of',
 '60',
 'kilos',
 'making',
 'a',
 'cumulative',
 'total',
 'for',
 'the',
 'season',
 'of',
 '5',
 '93',
 'mln',
 'against',
 '5',
 '81',
 'at',
 'the',
 'same',
 'stage',
 'last',
 'year',
 'Again',
 'it',
 'seems',
 'that',
 'cocoa',
 'delivered',
 'earlier',
 'on',
 'consignment',
 'was',
 'included',
 'in',
 'the',
 'arrivals',
 'figures',
 'Comissaria',
 'Smith',
 'said',
 'the

We now turn each document into a vector of W dimensions, where W is the number of words in the corpus. Each component of the vector represents the term frequency inverse document frequency (TF-IDF) of one word in the document. TF-IDF is a weighting scheme based on a word's occurrence in a document (term frequency) and its uniqueness in the overall corpus (inverse document frequency). A higher TF-IDF usually means the word is more important in the corpus.

There are [several different weighting schemes](https://en.wikipedia.org/wiki/Tf%E2%80%93idf#Definition) we could use; the ones I've chosen are as follows. 

For term frequency (tf) we will use a type of weighting scheme called [double normalization](https://nlp.stanford.edu/IR-book/html/htmledition/maximum-tf-normalization-1.html). The first normalization divides a word's raw count by the maximum raw count of a word in that document. This prevents TF's bias towards longer documents. Then we normalize again by "smoothing", which prevents modest changes in tf from greatly affecting TF-IDF.

For inverse document frequency (idf) we will use the standard logarithmically scaled frequency.


In [123]:
%%latex

$$ tf = 0.4 + (1-0.4)\frac{N(v_i, d)}{max_{v_j \in s}N(v_j, d)} $$

where $N(v_i, s)$ is the number of times word $v_i$ occurs in document $d$. We've used the common smoothing constant of 0.4.

$$ idf = log \frac{N}{N(v_i)} $$

where $N$ is the number of documents in the corpus, and $N(v_i)$ is the number of documents that includes $v_i$.

$$ TF\text{-}IDF = tf \times idf $$

<IPython.core.display.Latex object>

In [124]:
words = []
for doc in documents:
    words += doc
words = [w for w in set(words) if len(w) > 0]
numWords = len(words)

print('There are {} unique words over {} documents.'.format(numWords, len(documents)))

There are 11234 unique words over 925 documents.


In [137]:
wordToLoc = {words[i]: i for i in range(numWords)} # a word to index lookup

# stores number of documents a word appears in
documentCounts = [0 for _ in range(numWords)]

a = 0.4 # our smoothing constant

# Calculates tf vector while building document counts
def tf(doc):    
    vector = [0 for _ in range(numWords)]
    counted = defaultdict(bool) # keeps track of if we've counted the word towards document frequency
    for word in doc:
        if len(word) == 0:
            continue
        loc = wordToLoc[word]
        vector[loc] += 1
        if not counted[word]:
            documentCounts[loc] += 1
            counted[word] = True
    maxTf = max(vector)
    return [a + (1 - a) * c / maxTf for c in vector]

tfVectors = [tf(d) for d in documents]

In [148]:
idfCache = {}
def getIdf(i):
    if i not in idfCache:
        idfCache[i] = math.log10(len(documents) / documentCounts[i])
    return idfCache[i]

def normalizeByIdf(vector):
    output = []
    for i in range(len(vector)):
        output.append(vector[i] * getIdf(i))
    return output
            
tfidf = [normalizeByIdf(v) for v in tfVectors]